In [1]:
#Import different libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
import pickle


In [11]:
#Load the model
model = load_model('cnn_model.h5')

In [3]:
#Import paths
BASE_PATH = "../"
OLD_DATA = os.path.join(BASE_PATH, "data")
DATA_TRAIN = os.path.join(OLD_DATA, "train")
DATA_VALIDATION = os.path.join(OLD_DATA, "validation")
NEW_DATA = os.path.join(BASE_PATH, "new_data/train")

In [4]:
#define parameters
img_height, img_width = 150, 150
batch_size = 32
epochs = 30

In [5]:
#Split Data for validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)


In [6]:
# Generator for old data
old_train_generator = train_datagen.flow_from_directory(
    DATA_TRAIN,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

old_validation_generator = train_datagen.flow_from_directory(
    DATA_TRAIN,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 16933 images belonging to 4 classes.
Found 4232 images belonging to 4 classes.


In [7]:
# Générateur pour les nouvelles données
new_train_generator = train_datagen.flow_from_directory(
    NEW_DATA,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

new_validation_generator = train_datagen.flow_from_directory(
    NEW_DATA,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 16 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


In [8]:
#Combine generators
def combine_generators(*generators):
    while True:
        for generator in generators:
            yield next(generator)

#Combined gererator for training
mixed_train_generator = combine_generators(old_train_generator, new_train_generator)

#Combined gererator for validation
mixed_validation_generator = combine_generators(old_validation_generator, new_validation_generator)

#Adjust steps per epoch and validation steps
total_train_samples = old_train_generator.samples + new_train_generator.samples
total_val_samples = old_validation_generator.samples + new_validation_generator.samples

#Define steps per epoch
steps_per_epoch = total_train_samples // batch_size
validation_steps = total_val_samples // batch_size


In [21]:
#Recreate the optimizer
optimizer = tf.keras.optimizers.Adam()

#Compile the model with the new optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
#Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_incremental.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
]

In [19]:
#Continue training
history = model.fit(
    mixed_train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=mixed_validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/30
529/529 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.8110 - loss: 0.5108

InvalidArgumentError: Graph execution error:

Detected at node sequential_5_1/conv2d_16_1/add defined at (most recent call last):
  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\runpy.py", line 86, in _run_code

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\asyncio\windows_events.py", line 321, in run_forever

  File "c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\asyncio\base_events.py", line 603, in run_forever

 

In [ ]:
#Evaluate the model
loss, accuracy = model.evaluate(mixed_validation_generator, steps=validation_steps)
print(f"Validation accuracy: {accuracy * 100:.2f}%")

#Save the model
model.save('cnn_model_incremental.h5')

#Convert model to pickle
with open('cnn_model_incremental.pkl', 'wb') as file:
    pickle.dump(model, file)